<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN408_GenAI_671/blob/main/LangChain_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Tutorial

Material is based on https://www.youtube.com/watch?v=yF9kGESAi3M and https://python.langchain.com/docs/tutorials/chatbot/

## LangChain Basic

In [ ]:
#Colab already have langchain preinstalled
!pip install langchain

In [ ]:
!pip install -qU langchain-anthropic

LangChain supports many different language models

1) Claude Example

https://python.langchain.com/docs/integrations/chat/anthropic/

In [14]:
import os
from google.colab import userdata
from langchain_anthropic import ChatAnthropic

os.environ["ANTHROPIC_API_KEY"] = userdata.get('anthropic')

sonnet = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

In [15]:
sonnet.invoke("Hello")

AIMessage(content='Hi! How can I help you today?', additional_kwargs={}, response_metadata={'id': 'msg_01Ej6byqJYXFWUBDVvWNEeSe', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 8, 'output_tokens': 17}}, id='run-2189a071-8224-4021-ad45-d37a71ff687b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 17, 'total_tokens': 25, 'input_token_details': {}})

In [16]:
sonnet.invoke("Hello").content

'Hi! How can I help you today?'

2) OpenAI Example

In [8]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00


In [11]:
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

gpt4o = ChatOpenAI(model="gpt-4o")

In [12]:
gpt4o.invoke("Hello").content

'Hello! How can I assist you today?'

## Basic Conversation

In [21]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="Translate the following from English into Thai"),
    HumanMessage(content="hi!"),
]

sonnet.invoke(messages).content

'สวัสดีค่ะ/ครับ! (sawasdee ka/krap!)\n\nNote: ค่ะ (ka) is used by female speakers, ครับ (krap) is used by male speakers.'

In [20]:
gpt4o.invoke(messages).content

'สวัสดี!'

### Multi turn Conversation

In [23]:
messages = [
    SystemMessage(content="Translate the following from English into Thai"),
    HumanMessage(content="hi!"),
    AIMessage(content="สวัสดี! (sà-wàt-dii)"),
    HumanMessage(content="Thank you")
]

sonnet.invoke(messages).content

'ขอบคุณ (khàawp-khun)\n\nYou can also say:\n- ขอบคุณครับ (khàawp-khun khráp) - for men\n- ขอบคุณค่ะ (khàawp-khun khâ) - for women'

### Real-Time Conversation

In [ ]:
chat_history = []

system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)

# Chat loop
while True:
  query = input("You: ")
  if query == "exit":
    break

  chat_history.append(HumanMessage(content=query))

  result = sonnet.invoke(chat_history)
  response = result.content
  chat_history.append(AIMessage(content=response))

  print(f"AI: {response}")

print("---Message History---")
print(chat_history)

## Prompt Templates

Prompt templates = Strings with placeholders to be filled in

- Template = `"Generate {joke_number} jokes about {joke_topic}"`
- Input = `{"joke_number":3, "joke_topic":"cats"}`
- Output = `"Generate 3 jokes about cats"`

In [30]:
from langchain.prompts import ChatPromptTemplate

template = "Tell me a joke about {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.format_messages(topic="cats")
print(prompt)

[HumanMessage(content='Tell me a joke about cats.', additional_kwargs={}, response_metadata={})]


In [31]:
prompt = prompt_template.invoke({"topic":"cats"})
print(prompt)

messages=[HumanMessage(content='Tell me a joke about cats.', additional_kwargs={}, response_metadata={})]


In [33]:
sonnet.invoke(prompt).content

"Why don't cats like online shopping?\n\nThey prefer a cat-alog! 😺"

In [37]:
# Multiple placeholders

template_multiple = "Generate {joke_number} jokes about {joke_topic}"
prompt_template_multiple = ChatPromptTemplate.from_template(template_multiple)

prompt = prompt_template_multiple.invoke({"joke_number":3, "joke_topic":"cats"})
print(prompt)

messages=[HumanMessage(content='Generate 3 jokes about cats', additional_kwargs={}, response_metadata={})]


In [38]:
sonnet.invoke(prompt).content

"Here are 3 cat jokes:\n\n1. What do you call a cat that becomes a priest?\nA paw-ther!\n\n2. Why don't cats like online shopping?\nThey prefer a cat-alog!\n\n3. What do you call a cat that gets anything it wants?\nPurrsuasive!"

### Prompt Templates with System and Human Messages (Using Tuples)


In [35]:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human","Tell me {joke_number} jokes.")
]

prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"topic":"cats", "joke_number":3})
print(prompt)

messages=[SystemMessage(content='You are a comedian who tells jokes about cats.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes.', additional_kwargs={}, response_metadata={})]


In [36]:
sonnet.invoke(prompt).content

"Here are 3 cat-themed jokes:\n\n1. What do you call a cat that becomes a priest?\nA paw-ther!\n\n2. Why don't cats like online shopping?\nThey prefer a cat-alog!\n\n3. What's a cat's favorite dessert?\nA purr-fait!\n\n*adjusts microphone and waits for laughs*\n\nThese are just a small sample from my purr-sonal collection. I'll be here all week, folks! Remember to tip your waitstaff, and don't forget to spay and neuter your pets!"

In [39]:
sonnet.invoke(prompt_template.invoke({"topic":"Thai","joke_number":3})).content

'Here are 3 Thai-themed jokes:\n\n1. Why don\'t Thai people need alarm clocks?\nBecause they have a Thai-ming system!\n\n2. What do you call a confused Thai person in a restaurant?\nPad-Thai-ed and confused!\n\n3. What did the Thai chef say when he ran out of noodles?\n"This is a Pad situation!"\n\n*Note: These are light-hearted jokes meant to be fun and not offensive to Thai culture or people.'

## Chain

- Chain multiple invokable functions together in one command
- LangChain Expression Language (LECL)

### Basic Chain

In [40]:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human","Tell me {joke_number} jokes.")
]
prompt_template = ChatPromptTemplate.from_messages(messages)

chain = prompt_template | sonnet

result = chain.invoke({"topic":"cats", "joke_number":3})
print(result.content)

Here are 3 cat-themed jokes:

1. What do you call a cat that becomes a priest?
A paw-ther!

2. Why don't cats like online shopping?
They prefer a cat-alog!

3. What did the cat say when she lost all her money gambling?
I'm not feline lucky today!

*adjusts microphone and waits for laughs*


Langchain provides `StrOutputParser` function

In [41]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt_template | sonnet | StrOutputParser()

result = chain.invoke({"topic":"cats", "joke_number":3})
print(result)

Here are 3 cat-themed jokes:

1. What do you call a cat that becomes a priest?
A paw-ther!

2. Why don't cats like online shopping?
They prefer a cat-alog!

3. What did the cat say when he lost all his money gambling?
I'm not feline lucky today!

*adjusts microphone and waits for laughs* 
Hey, tough crowd tonight! These jokes are purr-fect, if you ask me!


### Under the hood
- You can create your own runnable functions (lambda).

In [42]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence

format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: sonnet.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

chain = RunnableSequence(first = format_prompt, middle= [invoke_model], last= parse_output)

result = chain.invoke({"topic":"cats", "joke_number":3})
print(result)

Here are 3 cat-themed jokes:

1. What do you call a cat that becomes a priest?
A paw-ther!

2. Why don't cats like online shopping?
They prefer a cat-alog!

3. What did the cat say when he lost all his money at the casino?
I'm not feline lucky today!

*adjusts microphone* 
Hey, if you think those were bad, you should hear my jokes about cat food - they're even worse! They really stink! *winks*


### Chain: Extended

In [43]:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human","Tell me {joke_number} jokes.")
]
prompt_template = ChatPromptTemplate.from_messages(messages)

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

chain = prompt_template | sonnet | StrOutputParser() | uppercase_output | count_words

result = chain.invoke({"topic":"cats", "joke_number":3})
print(result)

Word count: 77
HERE ARE 3 CAT-THEMED JOKES:

1. WHAT DO YOU CALL A CAT THAT BECOMES A PRIEST?
A PAW-THER!

2. WHY DON'T CATS LIKE ONLINE SHOPPING?
BECAUSE THEY PREFER A CAT-ALOG!

3. WHAT DO YOU CALL A CAT THAT GETS CAUGHT IN THE RAIN?
A DROWNED PURR-OUSE!

*ADJUSTS MICROPHONE AND WAITS FOR LAUGHS* 

THESE ARE JUST A SMALL SAMPLE FROM MY PURR-FESSIONAL REPERTOIRE. I'LL BE HERE ALL WEEK, FOLKS! REMEMBER TO TIP YOUR WAITSTAFF, AND TRY THE CATNIP!


### Chain: Parallel

- You can create chains with branching

In [54]:
from langchain.schema.runnable import RunnableParallel

# Product Review LLM
## Step:
## Get the features
## Two branching: 1) Pros and 2) Cons

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert product reviewer."),
        ("human","List the main features of the product {product_name}.")
    ]
)

def analyze_pros(features):
  pros_template = ChatPromptTemplate.from_messages(
      [
          ("system","You are an expert product reviewer."),
          ("human","Given these features: {features}, list the pros of these features")
      ]
  )
  return pros_template.format_prompt(features=features)

def analyze_cons(features):
  cons_template = ChatPromptTemplate.from_messages(
      [
          ("system","You are an expert product reviewer."),
          ("human","Given these features: {features}, list the cons of these features")
      ]
  )
  return cons_template.format_prompt(features=features)

# Simplify branches with LCEL
pros_branch = RunnableLambda(lambda x: analyze_pros(x)) | sonnet | StrOutputParser()
cons_branch = RunnableLambda(lambda x: analyze_cons(x)) | sonnet | StrOutputParser()

chain = (
    prompt_template
    | sonnet
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch, "cons": cons_branch})
    | RunnableLambda(lambda x: x["branches"]["pros"] + "\n\n" + x["branches"]["cons"])
)

print(chain.invoke({"product_name":"iPhone 15"}))

Here are the key pros of the iPhone 15's features:

Display Pros:
- Dynamic Island offers more intuitive notifications and live activities
- High-resolution OLED display provides excellent color accuracy and contrast
- Sharp resolution ensures crisp text and images
- Bright screen suitable for outdoor use

Camera System Pros:
- 48MP main camera enables higher detail capture and better low-light performance
- Improved Portrait mode for better depth effects and subject isolation
- Enhanced Night mode for superior low-light photography
- Versatile ultra-wide lens for landscape and architectural shots
- High-quality front camera for selfies and video calls

Performance Pros:
- A16 Bionic chip offers powerful performance for apps and games
- 6GB RAM ensures smooth multitasking
- Multiple storage options to suit different needs
- Capable of handling demanding tasks and future iOS updates

Design & Build Pros:
- Universal USB-C port compatibility with many devices and accessories
- Durable ae

### Chain : Multiple Branching

In [57]:
from langchain.schema.runnable import RunnableBranch

# Product Review LLM
## Step:
## Get the features
## Two branching: 1) Pros and 2) Cons

# Content Analysis

positive_feedback_tempalte = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human","Generate a thank you note for this positive feedback: {feedback}")
    ]
)

negative_feedback_tempalte = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human","Generate a response addressing this negative feedback: {feedback}")
    ]
)

neutral_feedback_tempalte = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human","Generate a request for more details for this neutral feedback: {feedback}")
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human","Generate a message to esclate this feedback to a human agent: {feedback}")
    ]
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
         "Classify this feedback into one of the following categories: positive, negative, neutral, or escalate: {feedback}.")
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
  (
      lambda x: 'positive' in x,
      positive_feedback_tempalte | sonnet | StrOutputParser()
  ),
  (
      lambda x: 'negative' in x,
      negative_feedback_tempalte | sonnet | StrOutputParser()
  ),
  (
      lambda x: 'neutral' in x,
      neutral_feedback_tempalte | sonnet | StrOutputParser()
  ),
  escalate_feedback_template | sonnet | StrOutputParser()
)

# Create the classification chain
classfication_chain = classification_template | sonnet | StrOutputParser()

# Combine classification and response generation into one chain
chain = classfication_chain | branches

# Run the chain with an example review
good_review = "The product is excellent. I really enjoyed using it and found it very helpful."
bad_review = "The product is terrible. It broke after just one use and the quality is very poor."
neutral_review = "The product is okay. It works as expected but nothing exceptional."
default = "I'm not sure about the product yet. Can you tell me more about its features and benefits."

result = chain.invoke({'feedback': good_review})
print(result)

Here's a thank you note for the positive feedback:

Dear Valued Customer,

Thank you so much for taking the time to share your wonderful feedback! We are thrilled to hear that you found our product to be excellent and very helpful. It brings us great joy to know that you really enjoyed your experience with us.

Your positive comments mean a lot to our team and validate our commitment to delivering quality products that make a difference for our customers. Feedback like yours motivates us to maintain our high standards and continue improving.

Thank you again for your support and kind words. We look forward to serving you again in the future!

Best regards,
[Company Name]


## RAG with LangChain
- Material from https://python.langchain.com/docs/integrations/vectorstores/pinecone/
- Note: Their docs is not good...

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [71]:
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('pinecone_key')

pc = Pinecone(api_key=pinecone_api_key)

In [80]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

Setup Embedding

In [73]:
!pip install -qU langchain-huggingface

In [81]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [82]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Manage Vector Store

#### Add items to vector store

In [83]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f680db97-e7f0-4844-bd03-855a3079d0de',
 'cee544e7-d843-4203-b5c6-bdc2c1d693f3',
 '5ad8baf7-d752-43b8-8650-6dcca2689aa9',
 'ec503d02-e094-422d-92ff-164af2055e79',
 'fbc5ca37-2f0a-4446-afbe-6e67251c0a93',
 'fb45a5de-12f9-4bea-9d6a-c15ebede3509',
 'c7bad5e5-e261-4075-bf86-da7ec36f90f6',
 '7985c3e1-acb9-466b-b408-cd11f640c2f3',
 '31b3a6ba-7d8d-4019-9627-284c26822fee',
 '36d96b1d-5195-48c8-93c9-2ca330ee5ce0']

### Query vector store

#### Query directly

In [84]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [85]:
# Similarity search with score
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.595264] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#### Query by turning into retriever

Transform the vector store into a retriever for easier usage in your chains.

In [94]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retrieved_docs = retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})
retrieved_docs

[Document(id='ec503d02-e094-422d-92ff-164af2055e79', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [96]:
retrieved_docs[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

### Chain with retriever

we'll use a prompt for RAG that is checked into the LangChain prompt hub ([here](https://smith.langchain.com/hub/rlm/rag-prompt)).

In [97]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [98]:
# Manually copy from the hub
prompt = ChatPromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:""")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nQuestion: filler question \n\nContext: filler context \n\nAnswer:", additional_kwargs={}, response_metadata={})]

In [99]:
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} # the dict with context and question is cast to a RunnableParallel.
    | prompt
    | sonnet
    | StrOutputParser()
)

for chunk in rag_chain.stream("Will it be hot tomorrow?"):
    print(chunk, end="", flush=True)

Based on the forecast, it will not be hot tomorrow, with temperatures reaching only 62 degrees Fahrenheit and cloudy conditions. This would be considered mild or cool weather for most regions.

As we've seen above, the input to `prompt` is expected to be a dict with keys `"context"` and `"question"`. So the first element of this chain builds runnables that will calculate both of these from the input question:

* `retriever | format_docs` passes the question through the retriever, generating Document objects, and then to `format_docs` to generate strings;
* `RunnablePassthrough()` passes through the input question unchanged.

More info here: https://python.langchain.com/docs/tutorials/rag/

## Tool Use with LangChain

- There are many tools. https://python.langchain.com/v0.1/docs/integrations/tools/

In [ ]:
!pip install --upgrade youtube_search
!pip install langchain-community langchain-core

- [Documentation](https://python.langchain.com/api_reference/_modules/langchain_community/tools/youtube/search.html#YouTubeSearchTool)
- YoutubeSearch: search for youtube videos associated with a person.

In [62]:
from langchain_community.tools import YouTubeSearchTool

youtube_tool = YouTubeSearchTool()

In [64]:
youtube_tool.run("Nutchanon Yongsatianchot")

"['https://www.youtube.com/watch?v=0ByyrBUcjqQ&pp=ygUYTnV0Y2hhbm9uIFlvbmdzYXRpYW5jaG90', 'https://www.youtube.com/watch?v=juAqT9LqmGI&pp=ygUYTnV0Y2hhbm9uIFlvbmdzYXRpYW5jaG90']"

In [67]:
# Bind the Youtube tool to the LLM
llm_with_tools = sonnet.bind_tools([youtube_tool])
msg = llm_with_tools.invoke("Nutchanon Yongsatianchot Youtube Videos")
print(msg)

content=[{'text': "I'll help you search for YouTube videos associated with Nutchanon Yongsatianchot.", 'type': 'text'}, {'id': 'toolu_013B32kjQv5MmqnhkdDQhLR9', 'input': {'query': 'Nutchanon Yongsatianchot'}, 'name': 'youtube_search', 'type': 'tool_use'}] additional_kwargs={} response_metadata={'id': 'msg_01Yb7A3s85WpasHyRWsJeFqV', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 435, 'output_tokens': 89}} id='run-496000eb-f9b3-4e44-b297-429a5017b528-0' tool_calls=[{'name': 'youtube_search', 'args': {'query': 'Nutchanon Yongsatianchot'}, 'id': 'toolu_013B32kjQv5MmqnhkdDQhLR9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 435, 'output_tokens': 89, 'total_tokens': 524, 'input_token_details': {}}


In [68]:
msg.tool_calls[0]["args"]["query"]

'Nutchanon Yongsatianchot'

In [70]:
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]["query"]) | youtube_tool
print(chain.invoke("Find some Youtube Videos about Nutchanon Yongsatianchot"))

['https://www.youtube.com/watch?v=0ByyrBUcjqQ&pp=ygUYTnV0Y2hhbm9uIFlvbmdzYXRpYW5jaG90', 'https://www.youtube.com/watch?v=juAqT9LqmGI&pp=ygUYTnV0Y2hhbm9uIFlvbmdzYXRpYW5jaG90']
